In [3]:
import os
import json
import redis
from redisgraph import Graph, Node, Edge
import matplotlib.pyplot as plt
import networkx as nx
from matplotlib.font_manager import FontProperties

font_path = "/Users/naokiyoshimaru/Downloads/ipaexg00401/ipaexg.ttf"
font_prop = FontProperties(fname=font_path)

In [4]:
graph_name = 'KyotoCityGraph'
redis_con = redis.Redis(host='localhost', port=6379)
graph = Graph(graph_name, redis_con)

In [5]:
query = """MATCH (n:SightseeingSpot) RETURN n.Name, n.Latitude, n.Longitude"""
result = graph.query(query)

# 結果を取得
stations = []
for record in result.result_set:
    station_name, lat, lon = record
    stations.append((station_name, lat, lon))


In [60]:
import folium

# 京都市の緯度経度
kyoto_lat, kyoto_lon = 35.0116, 135.7681

# 地図を作成
m = folium.Map(location=[kyoto_lat, kyoto_lon], zoom_start=12)

# stationsリストには前述のコードで取得した駅の緯度経度情報が入っていると仮定
for station_name, lat, lon in stations:
    folium.Marker([lat, lon], tooltip=station_name).add_to(m)
m.save('kyoto_SightseeingSpot_map.html')


In [23]:
def find_route_by_name(graph, sight1_name, sight2_name):
    # 最寄りの駅を見つけるクエリ
    query_nearest_stations = (
        f"MATCH (s1:SightseeingSpot)-[:NEAR]->(start:Station), "
        f"(s2:SightseeingSpot)-[:NEAR]->(end:Station) "
        f"WHERE s1.Name = '{sight1_name}' AND s2.Name = '{sight2_name}' "
        "RETURN start, end"
    )
    result = graph.query(query_nearest_stations)
    start_station, end_station = result.result_set[0]
    start_station_name, end_station_name = start_station.properties["Name"],end_station.properties["Name"]
    print(start_station_name, end_station_name)
    # 最短経路を見つけるクエリ
    query_shortest_path = f"MATCH path = (start:Station)-[:TRAINLINE*]-(end:Station) WHERE start.Name = '{start_station_name}' AND end.Name = '{end_station_name}' RETURN path"
    result = graph.query(query_shortest_path)
    return result.result_set[0][0]

# sight1_nameとsight2_nameに観光地の名前をセット
sight1_name = "角屋もてなしの文化美術館"
sight2_name = "藤森神社"
path = find_route_by_name(graph, sight1_name, sight2_name)
print(path)

丹波口駅 JR藤森駅
